In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.3)

from agents import Agent
from mouselab import MouselabEnv
from distributions import Normal, Categorical
from policies import FixedPlanPolicy

In [49]:
from bandits import MetaBanditEnv
from exact import solve
def parse(row):
    if row[0] < 0:
        return None
    row = row.tolist()
    arms = range(len(row) // 2)
    return tuple((row[a*2], row[a*2 + 1]) for a in arms)

template= '../matlab_code/GeneralMDP/states{n_arm}{max_obs}.csv'
for n_arm in range(2, 6):
    max_obs = 10
    cost = 0
    env = MetaBanditEnv(n_arm, max_obs, cost)
    Q, V, pi, info = solve(env)
    states = pd.read_csv(template.format(n_arm=n_arm, max_obs=max_obs))
    states = states.apply(parse, axis=1).iloc[:-1]
    
    def get_qs(s):
        if len(env.actions(s)) == 1:
            return [0] * n_arm + [Q(s, n_arm)]
        else:
            return [Q(s, a) for a in range(n_arm+1)]

    df = pd.DataFrame(np.stack(states.apply(get_qs)))
    df.to_csv('results/bandit_q_{n_arm}_{max_obs}.csv'.format(n_arm=n_arm, max_obs=max_obs),
              header=False, index=False)


In [89]:
from toolz.curried import *
# df.apply(compose(len, list, concat)).max()
# V(df.iloc[2])

0.7241666666666668

In [62]:


df = states.iloc[:10]
df.apply(parse, axis=1)

0    ((2, 1), (1, 1), (1, 1), (1, 1), (1, 1))
1    ((1, 2), (1, 1), (1, 1), (1, 1), (1, 1))
2    ((1, 1), (2, 1), (1, 1), (1, 1), (1, 1))
3    ((1, 1), (1, 2), (1, 1), (1, 1), (1, 1))
4    ((1, 1), (1, 1), (2, 1), (1, 1), (1, 1))
5    ((1, 1), (1, 1), (1, 2), (1, 1), (1, 1))
6    ((1, 1), (1, 1), (1, 1), (2, 1), (1, 1))
7    ((1, 1), (1, 1), (1, 1), (1, 2), (1, 1))
8    ((1, 1), (1, 1), (1, 1), (1, 1), (2, 1))
9    ((1, 1), (1, 1), (1, 1), (1, 1), (1, 2))
dtype: object